In [5]:
import boto3

# Create an EC2 client
ec2 = session.client('ec2')

# Create VPC
vpc_response = ec2.create_vpc(CidrBlock='172.31.0.0/16')
vpc_id = vpc_response['Vpc']['VpcId']

# Enable DNS support and DNS hostnames
ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={'Value': True})
ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={'Value': True})

# Tag the VPC
ec2.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'MyVPC'}])

{'ResponseMetadata': {'RequestId': 'a1f9b6ef-9b84-4166-8d73-7ebb3b3388f9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a1f9b6ef-9b84-4166-8d73-7ebb3b3388f9',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '221',
   'date': 'Tue, 18 Jun 2024 19:08:48 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [6]:

# Create Public Subnets
public_subnets = [
    {'CidrBlock': '172.31.1.0/24', 'AvailabilityZone': 'eu-west-1a', 'Tag': 'public_subnet_1'},
    {'CidrBlock': '172.31.2.0/24', 'AvailabilityZone': 'eu-west-1b', 'Tag': 'public_subnet_2'},
    {'CidrBlock': '172.31.3.0/24', 'AvailabilityZone': 'eu-west-1c', 'Tag': 'public_subnet_3'}
]

public_subnet_ids = []

for subnet in public_subnets:
    response = ec2.create_subnet(
        VpcId=vpc_id,
        CidrBlock=subnet['CidrBlock'],
        AvailabilityZone=subnet['AvailabilityZone']
    )
    subnet_id = response['Subnet']['SubnetId']
    public_subnet_ids.append(subnet_id)
    ec2.create_tags(Resources=[subnet_id], Tags=[{'Key': 'Name', 'Value': subnet['Tag']}])
    # Enable MapPublicIpOnLaunch
    ec2.modify_subnet_attribute(SubnetId=subnet_id, MapPublicIpOnLaunch={'Value': True})

# Create Private Subnets
private_subnets = [
    {'CidrBlock': '172.31.4.0/24', 'AvailabilityZone': 'eu-west-1a', 'Tag': 'private_subnet_1'},
    {'CidrBlock': '172.31.5.0/24', 'AvailabilityZone': 'eu-west-1b', 'Tag': 'private_subnet_2'},
    {'CidrBlock': '172.31.6.0/24', 'AvailabilityZone': 'eu-west-1c', 'Tag': 'private_subnet_3'}
]

private_subnet_ids = []

for subnet in private_subnets:
    response = ec2.create_subnet(
        VpcId=vpc_id,
        CidrBlock=subnet['CidrBlock'],
        AvailabilityZone=subnet['AvailabilityZone']
    )
    subnet_id = response['Subnet']['SubnetId']
    private_subnet_ids.append(subnet_id)
    ec2.create_tags(Resources=[subnet_id], Tags=[{'Key': 'Name', 'Value': subnet['Tag']}])

In [9]:
import boto3
from botocore.exceptions import ClientError

ec2 = boto3.client('ec2')

# Retrieve the VPC ID with the tag Name=MyVPC
vpcs = ec2.describe_vpcs(Filters=[{'Name': 'tag:Name', 'Values': ['MyVPC']}])
vpc_id = vpcs['Vpcs'][0]['VpcId']

# Function to check if a security group already exists
def get_security_group_id(group_name, vpc_id):
    try:
        response = ec2.describe_security_groups(
            Filters=[
                {'Name': 'group-name', 'Values': [group_name]},
                {'Name': 'vpc-id', 'Values': [vpc_id]}
            ]
        )
        return response['SecurityGroups'][0]['GroupId']
    except IndexError:
        return None

# Security Group name
group_name = 'MySecurityGroup'

# Check if the security group already exists
security_group_id = get_security_group_id(group_name, vpc_id)

if security_group_id:
    print(f"Security Group '{group_name}' already exists with ID: {security_group_id}")
else:
    # Create Security Group
    try:
        response = ec2.create_security_group(
            GroupName=group_name,
            Description='Allow specific ingress and egress traffic',
            VpcId=vpc_id
        )
        security_group_id = response['GroupId']

        # Define Ingress Rules
        ingress_rules = [
            {
                'IpProtocol': 'tcp',
                'FromPort': 443,
                'ToPort': 443,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 443,
                'ToPort': 443,
                'Ipv6Ranges': [{'CidrIpv6': '::/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 0,
                'ToPort': 65535,
                'IpRanges': [{'CidrIp': '87.254.251.87/32', 'Description': 'access from MC office'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'Ipv6Ranges': [{'CidrIpv6': '::/0', 'Description': 'SSH ip6'}]
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'Ipv6Ranges': [{'CidrIpv6': '::/0'}]
            }
        ]

        # Authorize Ingress Rules
        for rule in ingress_rules:
            ec2.authorize_security_group_ingress(
                GroupId=security_group_id,
                IpPermissions=[rule]
            )

        # Define Egress Rules
        egress_rules = [
            {
                'IpProtocol': 'tcp',
                'FromPort': 0,
                'ToPort': 65535,
                'Ipv6Ranges': [{'CidrIpv6': '::/0', 'Description': 'all out ipv6'}]
            }
        ]

        # Authorize Egress Rules
        for rule in egress_rules:
            ec2.authorize_security_group_egress(
                GroupId=security_group_id,
                IpPermissions=[rule]
            )

        print(f"Security Group '{group_name}' created with ID: {security_group_id}")

    except ClientError as e:
        print(f"An error occurred: {e}")

print(f"Security Group ID: {security_group_id}")



Security Group 'MySecurityGroup' created with ID: sg-06593dd185091409c
Security Group ID: sg-06593dd185091409c
